# 감성분석

In [27]:
# 영화 리뷰 데이터 불러오기
import pandas as pd

review_df = pd.read_csv('./movie_text_data/ratings.txt', header=0, sep="\t", quoting=3)
review_df.head(3)

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1


In [28]:
# 전처리
import re


review_df['document'] = review_df['document'].apply(lambda x : re.sub(pattern="[^ㄱ-ㅎ 가-힣 a-z A-Z]", repl=" ", string=str(x)))

review_df.head(10)

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...,1
2,4655635,폴리스스토리 시리즈는 부터 뉴까지 버릴께 하나도 없음 최고,1
3,9251303,와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,1
5,2190435,사랑을 해본사람이라면 처음부터 끝까지 웃을수 있는영화,1
6,9279041,완전 감동입니다 다시봐도 감동,1
7,7865729,개들의 전쟁 나오나요 나오면 빠로 보고 싶음,1
8,7477618,굿,1
9,9250537,바보가 아니라 병 쉰 인듯,1


In [29]:
#test - train set 나누기

from sklearn.model_selection import train_test_split

class_df = review_df['label']
feature_df = review_df.drop(['id', 'label'], axis=1, inplace=False)

X_train, X_test, y_train, y_test = train_test_split(feature_df, class_df, test_size=0.3, random_state=156)

X_train.shape, X_test.shape


((140000, 1), (60000, 1))

In [30]:
# 로지스틱 회귀 + CountVectorize

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

pipeline = Pipeline([
    ('cnt_vect', CountVectorizer(ngram_range=(1,2))),
    ('lr_clf',LogisticRegression(C=10))

])

pipeline.fit(X_train['document'], y_train)
pred = pipeline.predict(X_test['document'])
pred_probs = pipeline.predict_proba(X_test['document'])[:,1]

print('예측 정확도는 {0:.4f}, ROC_AUC는 {1:.4f}'.format(accuracy_score(y_test, pred), roc_auc_score(y_test, pred_probs)))

cnt = 0
for review in X_test['document']:
    if pred[cnt] == 1:
        sentiment = 'positive'
    else:
        sentiment = 'negative'

    cnt += 1
    if cnt == 100:
        break

    print('댓글 :{0} 는 {1}로 예상됩니다'.format(review, sentiment))


예측 정확도는 0.8116, ROC_AUC는 0.9028
댓글 :이름 함부러 짓는거 아니다 는 negative로 예상됩니다
댓글 : 점도 아깝다 쓰레기같은 영화 는 negative로 예상됩니다
댓글 :고어물론 최고다   기발한연출력   맘에들었음    는 positive로 예상됩니다
댓글 :이건 뭐 무섭지도 않고 는 negative로 예상됩니다
댓글 :다시는 중국이랑 같이 접목하지마라    쪽팔림 진심    중국을데려다가 이렇게밖에못써먹다니    는 negative로 예상됩니다
댓글 :꼴통 배우의 무작정 때려 부수고 아작내기  는 positive로 예상됩니다
댓글 :역시 판타지는 너무 잼있어 반지팬으로써   점 는 positive로 예상됩니다
댓글 :왜 평점이 이렇지     는 positive로 예상됩니다
댓글 :추억의 영화 는 positive로 예상됩니다
댓글 :배우에게 너무 의지하여 오히려 긴장감이 떨어지는 는 negative로 예상됩니다
댓글 :아  참내 볼일보고 안딱고 나온기분 현실성이 떨어지잔아 쌍팔년도에나 먹혔을법한 스토리였 는 negative로 예상됩니다
댓글 :월리스와 그루밋때부터 팬이여서   시리즈라 생각해 보았으나 난 이 스토리를 즐기기엔 너무 늙어버렸다  명절날 조카가 틀어놓으면 전부치며 볼듯  성인들이 볼 극장용 무비는 아님 는 negative로 예상됩니다
댓글 :혜진양  예전부터 팬으로서 다소 실망    는 negative로 예상됩니다
댓글 :음  우리 사랑이 때문에   는 positive로 예상됩니다
댓글 :극이 진행될수록 포스터에서 풍기는 일말의 기대감 흥미 을 무참히 날려버려주는 센스 는 negative로 예상됩니다
댓글 :글세 이건머 별로 무섭지도 않고 스토리도 영     몰입도도 별로인거같고 근데 왜이렇게 점수가 높은겨     는 negative로 예상됩니다
댓글 :어릴 적 꿈만 같았던 영화  아직도 교신 멜로디가 또렷이 생각난다  는 positive로 예상됩니다
댓글 :영화관에 내친구 둘이랑 나랑 진도나가려는 커플 이랑  명있

In [35]:
# 로지스틱 회귀 + TfidfVectorize

pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(ngram_range=(1,2))),
    ('lr_clf',LogisticRegression(C=10))

])

pipeline.fit(X_train['document'], y_train)
pred = pipeline.predict(X_test['document'])
pred_probs = pipeline.predict_proba(X_test['document'])[:,1]

print('예측 정확도는 {0:.4f}, ROC_AUC는 {1:.4f}'.format(accuracy_score(y_test, pred), roc_auc_score(y_test, pred_probs)))

예측 정확도는 0.8150, ROC_AUC는 0.9054


In [36]:
# 유튜브 댓글 데이터 불러오기 유튜브 댓글로 예측해보기

import re


with open('./crawling_result_text/gamst_comments.txt', mode='rt', encoding='utf-8') as f:
    strings = f.read()

strings = re.sub(pattern='[^ㄱ-ㅎ 가-힣 a-z A-Z \n]', repl='', string=strings)
comment_list = strings.split('\n')

pred_comment = pipeline.predict(comment_list)
pred_probs_score = pipeline.predict_proba(comment_list)[:,1]


sentiment_result = []

cnt = 0
for comment in comment_list:
    if pred_comment[cnt] == 1:
        sentiment = 'positive'
    else:
        sentiment = 'negative'

    pred_score = pred_probs_score[cnt]

    cnt += 1

    print('[{0}] 는 ({2:.4f}) 확률로 {1}로 예상됩니다'.format(comment, sentiment, pred_score))
    sentiment_result.append('[{0}] 는 ({2:.4f}) 확률로 {1}로 예상됩니다'.format(comment, sentiment, pred_score))


with open('./crawling_result_text/comment_sentiment_LR_Tfidf.txt', mode='wt', encoding='utf-8') as f:
    f.write("\n".join(sentiment_result))





[영상 좋아요댓글 한번씩 부탁드립니다] 는 (0.7770) 확률로 positive로 예상됩니다
[He is JJASTHIS] 는 (0.7166) 확률로 positive로 예상됩니다
[HE IS NOT CHANGMO] 는 (0.8046) 확률로 positive로 예상됩니다
[HE IS TALMO] 는 (0.7166) 확률로 positive로 예상됩니다
[He is Talmoalto] 는 (0.7166) 확률로 positive로 예상됩니다
[Bring rap JJASTHIS] 는 (0.5106) 확률로 positive로 예상됩니다
[ He Is gamshebellom] 는 (0.7166) 확률로 positive로 예상됩니다
[He is CODE GAMST] 는 (0.7587) 확률로 positive로 예상됩니다
[THISISZZASTHIS] 는 (0.4611) 확률로 negative로 예상됩니다
[ 여긴 진짜 개잘했네 ㅋㅋㅋㅋ] 는 (0.8237) 확률로 positive로 예상됩니다
[ he is punchline king] 는 (0.7881) 확률로 positive로 예상됩니다
[보여줬지] 는 (0.4654) 확률로 negative로 예상됩니다
[yo] 는 (0.2447) 확률로 negative로 예상됩니다
[shut the fxxk a man] 는 (0.7048) 확률로 positive로 예상됩니다
[ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ] 는 (0.4611) 확률로 negative로 예상됩니다
[느슨해진 한국힙합에 긴장감을 주는것 같다] 는 (0.6900) 확률로 positive로 예상됩니다
[  우결단의 인생 감스트] 는 (0.9299) 확률로 positive로 예상됩니다
[ 여기서부터 ㅈㄴ노래 좋음] 는 (0.9812) 확률로 positive로 예상됩니다
[ 개 중독된다 그냥 짜쓰디스 레게노] 는 (0.1906) 확률로 negative로 예상됩니다
[  지렸다 ] 는 (0.5210) 확률로 positive로 예상됩니다
[H

## 인공신경망

In [ ]:
# 영화 리뷰 데이터 불러오기

def read_data(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        # txt 파일의 헤더(id document label)는 제외하기
        data = data[1:]
    return data

train_data = read_data('./movie_text_data/ratings_train.txt')
test_data = read_data('./movie_text_data/ratings_test.txt')

print(len(test_data))

In [ ]:
#토큰화

from konlpy.tag import Okt
import json
import os
from pprint import pprint


okt = Okt()

def tokenize(doc):
    # norm은 정규화, stem은 근어로 표시하기를 나타냄
    return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]

if os.path.isfile('./crawling_result_text/train_docs.json'):
    with open('./crawling_result_text/train_docs.json') as f:
        train_docs = json.load(f)
    with open('./crawling_result_text/test_docs.json') as f:
        test_docs = json.load(f)
else:
    train_docs = [(tokenize(row[1]), row[2]) for row in train_data]
    test_docs = [(tokenize(row[1]), row[2]) for row in test_data]
    # JSON 파일로 저장
    with open('./crawling_result_text/train_docs.json', 'w', encoding="utf-8") as make_file:
        json.dump(train_docs, make_file, ensure_ascii=False, indent="\t")
    with open('./crawling_result_text/test_docs.json', 'w', encoding="utf-8") as make_file:
        json.dump(test_docs, make_file, ensure_ascii=False, indent="\t")

pprint(train_docs[0])

In [9]:
# CountVectorization
import nltk

tokens = [t for d in train_docs for t in d[0]]

text = nltk.Text(tokens, name='NMSC')
pprint(text.vocab().most_common(10))

selected_words = selected_words = [f[0] for f in text.vocab().most_common(10000)]

def term_frequency(doc):
    return [doc.count(word) for word in selected_words]

train_x = [term_frequency(d) for d, _ in train_docs]
test_x = [term_frequency(d) for d, _ in test_docs]
train_y = [c for _, c in train_docs]
test_y = [c for _, c in test_docs]

[('./Punctuation', 67778),
 ('영화/Noun', 50818),
 ('하다/Verb', 41209),
 ('이/Josa', 38540),
 ('보다/Verb', 38538),
 ('의/Josa', 30188),
 ('../Punctuation', 29055),
 ('가/Josa', 26627),
 ('에/Josa', 26468),
 ('을/Josa', 23118)]


In [17]:
import numpy as np

x_train = np.asarray(train_x).astype('float32')
x_test = np.asarray(test_x).astype('float32')

y_train = np.asarray(train_y).astype('float32')
y_test = np.asarray(test_y).astype('float32')

In [19]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
             loss=losses.binary_crossentropy,
             metrics=[metrics.binary_accuracy])

model.fit(x_train, y_train, epochs=10, batch_size=512)
results = model.evaluate(x_test, y_test)

Epoch 1/10
293/293 [==============================] - 165s 562ms/step - loss: 0.4521 - binary_accuracy: 0.8013
Epoch 2/10
293/293 [==============================] - 21s 72ms/step - loss: 0.3181 - binary_accuracy: 0.8629
Epoch 3/10
293/293 [==============================] - 14s 47ms/step - loss: 0.2924 - binary_accuracy: 0.8772
Epoch 4/10
293/293 [==============================] - 8s 28ms/step - loss: 0.2703 - binary_accuracy: 0.8891
Epoch 5/10
293/293 [==============================] - 8s 28ms/step - loss: 0.2552 - binary_accuracy: 0.8971
Epoch 6/10
293/293 [==============================] - 8s 28ms/step - loss: 0.2365 - binary_accuracy: 0.9062
Epoch 7/10
293/293 [==============================] - 8s 29ms/step - loss: 0.2130 - binary_accuracy: 0.9174
Epoch 8/10
293/293 [==============================] - 8s 28ms/step - loss: 0.1945 - binary_accuracy: 0.9260
Epoch 9/10
293/293 [==============================] - 8s 29ms/step - loss: 0.1766 - binary_accuracy: 0.9319
Epoch 10/10
1563/1563 [

In [26]:
# 유튜브 댓글 데이터 확인
import re


with open('./crawling_result_text/gamst_comments.txt', mode='rt', encoding='utf-8') as f:
    strings = f.read()

strings = re.sub(pattern='[^ㄱ-ㅎ 가-힣 a-z A-Z \n]', repl='', string=strings)
comment_list = strings.split('\n')


def predict_pos_neg(review):
    token = tokenize(review)
    tf = term_frequency(token)
    data = np.expand_dims(np.asarray(tf).astype('float32'), axis=0)
    score = float(model.predict(data))
    if(score > 0.5):
        print("[{}]는 {:.2f}% 확률로 긍정 리뷰이지 않을까 추측해봅니다.^^\n".format(review, score * 100))
        return "[{}]는 {:.2f}% 확률로 긍정 리뷰이지 않을까 추측해봅니다.^^\n".format(review, score * 100)
    else:
        print("[{}]는 {:.2f}% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;\n".format(review, (1 - score) * 100))
        return "[{}]는 {:.2f}% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;\n".format(review, (1 - score) * 100)


sentiment_result = []


for comment in comment_list:
    sentiment_result.append(predict_pos_neg(comment))

with open('./crawling_result_text/comment_sentiment_ML.txt', mode='wt', encoding='utf-8') as f:
    f.write("\n".join(sentiment_result))

[영상 좋아요댓글 한번씩 부탁드립니다]는 96.29% 확률로 긍정 리뷰이지 않을까 추측해봅니다.^^

[He is JJASTHIS]는 72.57% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;

[HE IS NOT CHANGMO]는 51.59% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;

[HE IS TALMO]는 51.59% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;

[He is Talmoalto]는 72.57% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;

[Bring rap JJASTHIS]는 51.59% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;

[ He Is gamshebellom]는 51.59% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;

[He is CODE GAMST]는 72.57% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;

[THISISZZASTHIS]는 51.59% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;

[ 여긴 진짜 개잘했네 ㅋㅋㅋㅋ]는 54.40% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;

[ he is punchline king]는 72.57% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;

[보여줬지]는 60.36% 확률로 긍정 리뷰이지 않을까 추측해봅니다.^^

[yo]는 51.59% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;

[shut the fxxk a man]는 62.16% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;

[ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ]는 72.22% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;

[느슨해진 한국힙합에 긴장감을 주는것 같다]는 83.72% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;

[  우결단의 인생 감스트]는 75.66% 확률로 긍정 리뷰이지 않을까 추측해봅니다.^^

[ 여기서부터 ㅈㄴ노래 좋음]는 97.25% 확률로 긍정 리뷰이지 않을까 추측해봅니다.^^

[ 개 중독된다 